# Рекомендательные системы
## Programming assignment

***Oписание задачи:***

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать **recall@k** и **precision@k**.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

In [1]:
import pandas as pd
import numpy as np
from __future__ import division

In [2]:
train_data = pd.read_csv('coursera_sessions_train.txt', sep=';', names=['viewed', 'purchased'])
test_data = pd.read_csv('coursera_sessions_test.txt', sep=';', names=['viewed', 'purchased'])

***Входные данные***

Вам дается две выборки с пользовательскими сессиями - **id**-шниками просмотренных и **id**-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: **id** просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, **1,2,3,4**; или **1,2,3,4;5,6**.

Гарантируется, что среди **id** купленных товаров все различные.

***Важно:***

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k / precision@k.**

**Задание:**

1. На обучении постройте частоты появления **id** в просмотренных и в купленных (**id** может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:
    - сортировка просмотренных **id** по популярности (частота появления в просмотренных),
    - сортировка просмотренных **id** по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [134]:
from collections import OrderedDict


viewed = OrderedDict()
for session in train_data['viewed']:
    for product in session.split(','):
        viewed[product] = viewed.get(product, 0) + 1 
                   
purchased = OrderedDict()
for session in train_data.dropna()['purchased']:
    for product in session.split(','):
        purchased[product] = purchased.get(product, 0) + 1

for session in test_data['viewed'].values:
    for product in session.split(','):
        if product not in viewed:
            viewed[product] = 0
        elif product not in purchased:
            purchased[product] = 0

In [6]:
is_viewed_train = []
for session in train_data['viewed'].values:
    for product in session.split(','):
        is_viewed_train.append(product)

In [139]:
is_viewed_test = []
for session in test_data['viewed'].values:
    for product in session.split(','):
        is_viewed_test.append(product)

In [140]:
for product in is_viewed_train:
    if product not in purchased:
        purchased[product] = 0

NameError: name 'is_viewed_train' is not defined

In [23]:
train_order = dict()
for session in train_data['viewed'].values:
    idx = 20
    for product in session.split(','):
        if product not in train_order:
            train_order[product] = idx
            idx -= 1

In [141]:
for product in is_viewed_test:
    if product not in purchased:
        purchased[product] = 0
    elif product not in test_order:
        test_order[product] = 0

In [12]:
sorted_train_viewed = []
for idx in train_data.dropna()['viewed'].values:
    sorted_train_viewed.append(sorted(set(idx.split(',')), key=lambda idx: viewed[idx], reverse=True))

In [18]:
sorted_train_purchased = []
for idx in train_data.dropna()['viewed'].values:
    sorted_train_purchased.append(sorted(set(idx.split(',')), key=lambda idx: (purchased[idx], train_order[idx]), reverse=True))

In [142]:
sorted_test_purchased = []
for idx in test_data.dropna()['viewed'].values:
    sorted_test_purchased.append(sorted(set(idx.split(',')), key=lambda idx: (purchased[idx], test_order[idx]), reverse=True))

In [20]:
sorted_test_viewed = []
for idx in test_data.dropna()['viewed'].values:
    sorted_test_viewed.append(sorted(set(idx.split(',')), key=lambda idx: viewed[idx], reverse=True))

In [135]:
test_order = dict()
for session in test_data['viewed'].values:
    idx = 100
    for product in session.split(','):
        if product not in test_order:
            test_order[product] = idx
            idx -= 1

In [138]:
purchased['69']

KeyError: '69'

In [18]:
def precision_k(sorted_list, data, k):
    answer_list = []
    for viewed, purchased in zip(sorted_list, data.dropna()['purchased'].values):
        res = len(set(viewed[:k]) & set(purchased.split(','))) / k
        answer_list.append(res)
    return round(np.mean(answer_list), 2)

In [19]:
def recall_k(sorted_list, data, k):
    answer_list = []
    for viewed, purchased in zip(sorted_list, data.dropna()['purchased'].values):
        res = len(set(viewed[:k]) & set(purchased.split(','))) / len(purchased.split(','))
        answer_list.append(res)
    return round(np.mean(answer_list), 2)

In [20]:
def write_answer_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(' '.join(str(num) for num in answer))

In [24]:
answer_1 = recall_k(sorted_train_viewed, train_data, k=1), precision_k(sorted_train_viewed, train_data, k=1), \
           recall_k(sorted_train_viewed, train_data, k=5), precision_k(sorted_train_viewed, train_data, k=5)

In [25]:
answer_2 = recall_k(sorted_test_viewed, test_data, k=1), precision_k(sorted_test_viewed, test_data, k=1), \
           recall_k(sorted_test_viewed, test_data, k=5), precision_k(sorted_test_viewed, test_data, k=5)

In [26]:
answer_3 = recall_k(sorted_train_purchased, train_data, k=1), precision_k(sorted_train_purchased, train_data, k=1), \
           recall_k(sorted_train_purchased, train_data, k=5), precision_k(sorted_train_purchased, train_data, k=5)

In [144]:
answer_4 = recall_k(sorted_test_purchased, test_data, k=1), precision_k(sorted_test_purchased, test_data, k=1), \
           recall_k(sorted_test_purchased, test_data, k=5), precision_k(sorted_test_purchased, test_data, k=5)

In [145]:
answer_4

(0.43, 0.49, 0.8, 0.2)

In [73]:
answer_4

(0.43, 0.49, 0.8, 0.2)

In [161]:
answer_4

(0.57, 0.66, 0.89, 0.24)

In [146]:
sorted_test_purchased

[['63', '68', '69', '70', '66', '61', '59'],
 ['158', '162', '160', '159', '161'],
 ['204', '202', '200', '201', '203'],
 ['371', '372'],
 ['422'],
 ['469',
  '474',
  '463',
  '465',
  '466',
  '19',
  '467',
  '464',
  '468',
  '470',
  '471',
  '472',
  '473'],
 ['544', '545', '543', '546'],
 ['905',
  '906',
  '901',
  '900',
  '894',
  '904',
  '902',
  '895',
  '896',
  '907',
  '908',
  '909',
  '910',
  '911',
  '912',
  '913',
  '914',
  '898',
  '915',
  '916',
  '917'],
 ['1063'],
 ['1119', '1120', '1121', '1122', '1123', '1124', '1125', '1126'],
 ['1174', '1175', '500', '1176', '1177'],
 ['1262', '1260', '1265', '1257', '1258', '1259', '1261', '1263', '1264'],
 ['158',
  '162',
  '160',
  '551',
  '1282',
  '1283',
  '1278',
  '1279',
  '1280',
  '161',
  '1281',
  '1245'],
 ['1328', '1334', '1335', '1336'],
 ['1374'],
 ['1493'],
 ['1563'],
 ['1657', '1658'],
 ['1678',
  '1673',
  '1674',
  '1675',
  '1676',
  '1677',
  '213',
  '1679',
  '1680',
  '1681',
  '1682',
  '1683

In [50]:
#write_answer_to_file(answer_1, 'views_popularity_train.txt')
#write_answer_to_file(answer_2, 'views_popularity_test.txt')
#write_answer_to_file(answer_3, 'purchases_popularity_train.txt')
write_answer_to_file(answer_4, 'purchases_popularity_test.txt')

In [51]:
purchased['472']

0

In [105]:
answer_list_5 = []
for viewed, purchased in zip(sorted_test_purchased, test_data.dropna()['purchased'].values):
    res = len(set(viewed[:5]) & set(purchased.split(','))) / 5
    answer_list_5.append(res)

In [106]:
answer_list_1 = []
for viewed, purchased in zip(sorted_test_purchased, test_data.dropna()['purchased'].values):
    res = len(set(viewed[:1]) & set(purchased.split(',')))
    answer_list_1.append(res)

In [107]:
answer_list_6 = []
for viewed, purchased in zip(sorted_test_purchased, test_data.dropna()['purchased'].values):
    res = len(set(viewed[:5]) & set(purchased.split(','))) / len(purchased.split(','))
    answer_list_6.append(res)

In [108]:
answer_list_2 = []
for viewed, purchased in zip(sorted_test_purchased, test_data.dropna()['purchased'].values):
    res = len(set(viewed[:1]) & set(purchased.split(','))) / len(purchased.split(','))
    answer_list_2.append(res)

In [118]:
np.array(answer_list_5).sum() / len(answer_list_5)

0.20458390177353344

In [28]:
test_data.dropna().head()

,viewed,purchased
7,"63,68,69,70,66,61,59,61,66,68","66,63"
14,"158,159,160,159,161,162",162
19,"200,201,202,203,204","201,205"
34,"371,372,371","371,373"
40,422,422


In [79]:
purchased_1['161']

0

In [85]:
sorted_test_purchased_1

[['63', '68', '69', '70', '66', '61', '59'],
 ['158', '162', '160', '159', '161'],
 ['204', '202', '200', '201', '203'],
 ['371', '372'],
 ['422'],
 ['469',
  '474',
  '463',
  '465',
  '466',
  '19',
  '467',
  '464',
  '468',
  '470',
  '471',
  '472',
  '473'],
 ['544', '545', '543', '546'],
 ['905',
  '906',
  '901',
  '900',
  '894',
  '904',
  '902',
  '895',
  '896',
  '907',
  '908',
  '909',
  '910',
  '911',
  '912',
  '913',
  '914',
  '898',
  '915',
  '916',
  '917'],
 ['1063'],
 ['1119', '1120', '1121', '1122', '1123', '1124', '1125', '1126'],
 ['1174', '1175', '500', '1176', '1177'],
 ['1262', '1260', '1265', '1257', '1258', '1259', '1261', '1263', '1264'],
 ['158',
  '162',
  '160',
  '551',
  '1282',
  '1283',
  '1278',
  '1279',
  '1280',
  '161',
  '1281',
  '1245'],
 ['1328', '1334', '1335', '1336'],
 ['1374'],
 ['1493'],
 ['1563'],
 ['1657', '1658'],
 ['1678',
  '1673',
  '1674',
  '1675',
  '1676',
  '1677',
  '213',
  '1679',
  '1680',
  '1681',
  '1682',
  '1683

In [70]:
sorted_test_purchased_1 = []
for idx in test_data.dropna()['viewed'].values:
    sorted_test_purchased_1.append(sorted(set(idx.split(',')), key=lambda idx: (purchased_1[idx],
                                                                                test_order[idx]), reverse=True))